In [126]:
import gensim
import os
import collections
import random
import pandas as pd

In [128]:
data = pd.read_csv('../data/man.csv.gz', delimiter='\t', compression='gzip')
data.data = data.data.apply(str)

In [232]:
texts = [[word for word in gensim.utils.simple_preprocess(document)]
         for document in data['data']]


In [233]:
docs = []
for i, text in enumerate(texts):
    docs.append(gensim.models.doc2vec.TaggedDocument(text, [i]))
    
    

In [238]:
model = gensim.models.doc2vec.Doc2Vec(size=100, min_count=5, iter=10, workers=4)

In [239]:
train_corpus = docs


In [240]:
model.build_vocab(train_corpus)

In [241]:
%time model.train(train_corpus)

CPU times: user 13min 31s, sys: 712 ms, total: 13min 32s
Wall time: 3min 26s


112493020

In [138]:
def get_index(name):
    for i, d in enumerate(train_corpus):
        if d.words[0] == name:
            return i
    return 0

ls = get_index('malloc')
inferred = model.infer_vector(train_corpus[ls].words)

In [139]:
sims = model.docvecs.most_similar([inferred], topn=10)

In [140]:
sims

[(2532, 0.7567048668861389),
 (2652, 0.7403140664100647),
 (2527, 0.736198365688324),
 (3142, 0.7285833358764648),
 (2927, 0.726046085357666),
 (3428, 0.7226648330688477),
 (3296, 0.7077306509017944),
 (2276, 0.6990418434143066),
 (3125, 0.6801280379295349),
 (2223, 0.6480376720428467)]

In [323]:
def get_most_similar(name, section=None):
    if section is None:
        index = np.where(data['name'] == name)[0][0]
    else:
        index = np.where((data['name'] == name) & (data['section'] == section))[0][0]
        
    inferred = model.infer_vector(train_corpus[index].words)
    sims = model.docvecs.most_similar([inferred], topn=10)
    for i,score in sims:
        print(data.name.iloc[i], score)


In [329]:
get_most_similar('ping')

ping 0.8317750692367554
ping6 0.7819598913192749
traceroute 0.6307887434959412
rdisc 0.5278071761131287
ipsend 0.5260852575302124
mtrace 0.525686502456665
dhcrelay 0.5218509435653687
dhcpcd 0.5140422582626343
pcap-filter 0.5051119923591614
ifconfig 0.5018467903137207


In [279]:
def search(query):
    inferred = model.infer_vector(gensim.utils.simple_preprocess(query))
    sims = model.docvecs.most_similar([inferred], topn=10)
    for i,score in sims:
        print(data.name.iloc[i], score)


In [285]:
search('')

VFS_ROOT 0.407088965177536
vfsops 0.4043636918067932
VFS_NEWVNODE 0.4014365077018738
VFS_FHTOVP 0.3968418836593628
VFS_SNAPSHOT 0.39564093947410583
FcFileIsDir 0.3724491000175476
fstrans 0.3722206950187683
VFS_QUOTACTL 0.36890166997909546
VFS_STATVFS 0.36708885431289673
rmextattr 0.36662667989730835


In [330]:
model.save('./man.doc2vec')